# FINN - Functional Verification of End-to-End Flow
-----------------------------------------------------------------

**Important: This notebook depends on the tfc_end2end_example notebook, because we are using models that are available at intermediate steps in the end-to-end flow. So please make sure the needed .onnx files are generated to run this notebook.**

In this notebook, we will show how to take the intermediate results of the end-to-end tfc example and verify their functionality with different methods. In the following picture you can see the section in the end-to-end flow about the *Simulation & Emulation Flows*. Besides the methods in this notebook, there is another one that is covered in the Jupyter notebook [tfc_end2end_example](tfc_end2end_example.ipynb): remote execution. The remote execution allows functional verification directly on the PYNQ board, for details please have a look at the mentioned Jupyter notebook.

<img src="verification.png" alt="Drawing" style="width: 500px;"/>

We will use the following helper functions, `showSrc` to show source code of FINN library calls and `showInNetron` to show the ONNX model at the current transformation step. The Netron displays are interactive, but they only work when running the notebook actively and not on GitHub (i.e. if you are viewing this on GitHub you'll only see blank squares).

In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showSrc, showInNetron
   
build_dir = "/workspace/finn"

To verify the simulations, a "golden" output is calculated as a reference. This is calculated directly from the Brevitas model using PyTorch, by running some example data from the MNIST dataset through the trained model.

In [2]:
from pkgutil import get_data
import onnx
import onnx.numpy_helper as nph
import torch
from finn.util.test import get_test_model_trained

fc = get_test_model_trained("TFC", 1, 1)
raw_i = get_data("finn.data", "onnx/mnist-conv/test_data_set_0/input_0.pb")
input_tensor = onnx.load_tensor_from_string(raw_i)
input_brevitas = torch.from_numpy(nph.to_array(input_tensor)).float()
output_golden = fc.forward(input_brevitas).detach().numpy()
output_golden

<ipython-input-2-ddb8addf4232>:10: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  input_brevitas = torch.from_numpy(nph.to_array(input_tensor)).float()


array([[-1.4090618, -1.3267527,  0.9779036, -1.2444434, -1.4090618,
        -1.6559892, -1.3267527, -1.4913709, -1.3267527, -1.6559892]],
      dtype=float32)

## Simulation using Python <a id='simpy'></a>

If an ONNX model consists of [standard ONNX](https://github.com/onnx/onnx/blob/master/docs/Operators.md) nodes and/or FINN custom operations that do not belong to the fpgadataflow (backend $\neq$ "fpgadataflow") this model can be checked for functionality using Python.

To simulate a standard ONNX node [onnxruntime](https://github.com/microsoft/onnxruntime) is used. onnxruntime is an open source tool developed by Microsoft to run standard ONNX nodes. For the FINN custom op nodes execution functions are defined. The following is an example of the execution function of a XNOR popcount node.


In [3]:
from finn.custom_op.general.xnorpopcount import xnorpopcountmatmul
showSrc(xnorpopcountmatmul)

def xnorpopcountmatmul(inp0, inp1):
    """Simulates XNOR-popcount matrix multiplication as a regular bipolar
    matrix multiplication followed by some post processing."""
    # extract the operand shapes
    # (M, K0) = inp0.shape
    # (K1, N) = inp1.shape
    K0 = inp0.shape[-1]
    K1 = inp1.shape[0]
    # make sure shapes are compatible with matmul
    assert K0 == K1, "Matrix shapes are not compatible with matmul."
    K = K0
    # convert binary inputs to bipolar
    inp0_bipolar = 2.0 * inp0 - 1.0
    inp1_bipolar = 2.0 * inp1 - 1.0
    # call regular numpy matrix multiplication
    out = np.matmul(inp0_bipolar, inp1_bipolar)
    # XNOR-popcount does not produce the regular dot product result --
    # it returns the number of +1s after XNOR. let P be the number of +1s
    # and N be the number of -1s. XNOR-popcount returns P, whereas the
    # regular dot product result from numpy is P-N, so we need to apply
    # some correction.
    # out = P-N
    # K = P+N
    # out + K = 

The function contains a description of the behaviour in Python and can thus calculate the result of the node.

This execution function and onnxruntime is used when `execute_onnx` from `onnx_exec` is applied to the model. The model is then simulated node by node and the result is stored in a context dictionary, which contains the values of each tensor at the end of the execution. To get the result, only the output tensor has to be extracted.

The procedure is shown below. We take the model right before the nodes should be converted into HLS layers and generate an input tensor to pass to the execution function. The input tensor is generated from the Brevitas example inputs.

In [4]:
import numpy as np
from finn.core.modelwrapper import ModelWrapper
input_dict = {"global_in": nph.to_array(input_tensor)}

model_for_sim = ModelWrapper(build_dir+"/tfc_w1a1_ready_for_hls_conversion.onnx")

In [5]:
import finn.core.onnx_exec as oxe
output_dict = oxe.execute_onnx(model_for_sim, input_dict)
output_pysim = output_dict[list(output_dict.keys())[0]]



if np.isclose(output_pysim, output_golden, atol=1e-3).all():
    print("Results are the same!")
else:
    print("The results are not the same!")

The results are not the same!


The result is compared with the theoretical "golden" value for verification.

## Simulation (cppsim) using C++

When dealing with HLS custom op nodes in FINN the simulation using Python is no longer sufficient. After the nodes have been converted to HLS layers, the simulation using C++ can be used. To do this, the input tensor is stored in an .npy file and C++ code is generated that reads the values from the .npy array, streams them to the corresponding finn-hlslib function and writes the result to a new .npy file. This in turn can be read in Python and processed in the FINN flow. For this example the model after setting the folding factors in the HLS layers is used, please be aware that this is not the full model, but the dataflow partition, so before executing at the end of this section we have to integrate the model back into the parent model.

In [6]:
model_for_cppsim = ModelWrapper(build_dir+"/tfc_w1_a1_set_folding_factors.onnx")

To generate the code for this simulation and to generate the executable two transformations are used:
* `PrepareCppSim` which generates the C++ code for the corresponding hls layer
* `CompileCppSim` which compules the C++ code and stores the path to the executable

In [7]:
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.general import GiveUniqueNodeNames

model_for_cppsim = model_for_cppsim.transform(GiveUniqueNodeNames())
model_for_cppsim = model_for_cppsim.transform(PrepareCppSim())
model_for_cppsim = model_for_cppsim.transform(CompileCppSim())

When we take a look at the model using netron, we can see that the transformations introduced new attributes.

In [9]:
model_for_cppsim.save(build_dir+"/tfc_w1_a1_for_cppsim.onnx")
showInNetron(build_dir+"/tfc_w1_a1_for_cppsim.onnx")

Serving '/workspace/finn/tfc_w1_a1_for_cppsim.onnx' at http://0.0.0.0:8081


The following node attributes have been added:
* `code_gen_dir_cppsim` indicates the directory where the files for the simulation using C++ are stored
* `executable_path` specifies the path to the executable

We take now a closer look into the files that were generated:

In [10]:
from finn.custom_op.registry import getCustomOp

fc0 = model_for_cppsim.graph.node[1]
fc0w = getCustomOp(fc0)
code_gen_dir = fc0w.get_nodeattr("code_gen_dir_cppsim")
!ls {code_gen_dir}

compile.sh			    memblock_0.dat  thresh.h
execute_StreamingFCLayer_Batch.cpp  node_model	    weights.npy


Besides the .cpp file, the folder contains .h files with the weights and thresholds. The shell script contains the compile command and *node_model* is the executable generated by compilation. Comparing this with the `executable_path` node attribute, it can be seen that it specifies exactly the path to *node_model*.

To simulate the model the execution mode(exec_mode) must be set to "cppsim". This is done using the transformation SetExecMode.

In [14]:
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

model_for_cppsim = model_for_cppsim.transform(SetExecMode("cppsim"))
model_for_cppsim.save(build_dir+"/tfc_w1_a1_for_cppsim.onnx")

Before the model can be executed using `execute_onnx`, we integrate the child model in the parent model. The function reads then the `exec_mode` and writes the input into the correct directory in a .npy file. To be able to read this in C++, there is an additional .hpp file ([npy2apintstream.hpp](https://github.com/Xilinx/finn/blob/master/src/finn/data/cpp/npy2apintstream.hpp)) in FINN, which uses cnpy to read .npy files and convert them into streams, or to read a stream and write it into an .npy. [cnpy](https://github.com/rogersce/cnpy) is a helper to read and write .npy and .npz formates in C++.

The result is again compared to the "golden" output.

In [17]:
parent_model = ModelWrapper(build_dir+"/tfc_w1_a1_dataflow_parent.onnx")
sdp_node = parent_model.graph.node[1]
child_model = build_dir + "/tfc_w1_a1_for_cppsim.onnx"
getCustomOp(sdp_node).set_nodeattr("model", child_model)
output_dict = oxe.execute_onnx(parent_model, input_dict)
output_cppsim = output_dict[list(output_dict.keys())[0]]

if np.isclose(output_cppsim, output_golden, atol=1e-3).all():
    print("Results are the same!")
else:
    print("The results are not the same!")

The results are not the same!


## Emulation (rtlsim) using PyVerilator

The emulation using [PyVerilator](https://github.com/maltanar/pyverilator) can be done after IP blocks are generated from the corresponding HLS layers. Pyverilator is a tool which makes it possible to simulate verilog files using verilator via a python interface.

We have two ways to use rtlsim, one is to run the model node-by-node as with the simulation methods, but if the model is in the form of the dataflow partition, the part of the graph that consist of only HLS nodes could also be executed as whole.

Because at the point where we want to grab and verify the model, the model is already in split form (parent graph consisting of non-hls layers and child graph consisting only of hls layers) we first have to reference the child graph within the parent graph. This is done using the node attribute `model` for the `StreamingDataflowPartition` node.

First the procedure is shown, if the child graph has ip blocks corresponding to the individual layers, then the procedure is shown, if the child graph already has a stitched IP.

### Emulation of model node-by-node

The child model is loaded and the `exec_mode` for each node is set. To prepare the node-by-node emulation the transformation `PrepareRTLSim` is applied to the child model. With this transformation the emulation files are created for each node and can be used directly when calling `execute_onnx()`. Each node has a new node attribute "rtlsim_so" after transformation, which contains the path to the corresponding emulation files. Then it is saved in a new .onnx file so that the changed model can be referenced in the parent model.

In [18]:
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP

test_fpga_part = "xc7z020clg400-1"
target_clk_ns = 10

child_model = ModelWrapper(build_dir + "/tfc_w1_a1_set_folding_factors.onnx")
child_model = child_model.transform(GiveUniqueNodeNames())
child_model = child_model.transform(PrepareIP(test_fpga_part, target_clk_ns))
child_model = child_model.transform(HLSSynthIP())
child_model = child_model.transform(SetExecMode("rtlsim"))
child_model = child_model.transform(PrepareRTLSim())
child_model.save(build_dir + "/tfc_w1_a1_dataflow_child.onnx")

%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch_0.v:65: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: Use "/* verilator lint_off STMTDLY */" and lint_on around source to disable this message.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch_0.v:66: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch.v:68: Unsupported: Ignoring delay on this delayed statement.
%Warning-WIDTH: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch.v:183: Operator ASSIGN expects 3 bits on the As

%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_1_96qy64f6/project_StreamingFCLayer_Batch_1/sol1/impl/verilog//StreamingFCLayer_Batch_1_StreamingFCLayer_Batch_1.v:78: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: Use "/* verilator lint_off STMTDLY */" and lint_on around source to disable this message.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_1_96qy64f6/project_StreamingFCLayer_Batch_1/sol1/impl/verilog//StreamingFCLayer_Batch_1_StreamingFCLayer_Batch_1.v:79: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_1_96qy64f6/project_StreamingFCLayer_Batch_1/sol1/impl/verilog//StreamingFCLayer_Batch_1_Matrix_Vector_Activa.v:601: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_1_96qy64f6/project_StreamingFCLayer_Batch_1/sol1/impl/verilog//StreamingFCLay

%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_3_rb8vkqnn/project_StreamingFCLayer_Batch_3/sol1/impl/verilog//StreamingFCLayer_Batch_3_StreamingFCLayer_Batch_3.v:78: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: Use "/* verilator lint_off STMTDLY */" and lint_on around source to disable this message.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_3_rb8vkqnn/project_StreamingFCLayer_Batch_3/sol1/impl/verilog//StreamingFCLayer_Batch_3_StreamingFCLayer_Batch_3.v:79: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_3_rb8vkqnn/project_StreamingFCLayer_Batch_3/sol1/impl/verilog//StreamingFCLayer_Batch_3_Matrix_Vector_Activa.v:663: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_3_rb8vkqnn/project_StreamingFCLayer_Batch_3/sol1/impl/verilog//StreamingFCLay

make: Entering directory '/tmp/finn_dev_zqc/pyverilator_LabelSelect_Batch_0_utzy75nd'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_zqc/pyverilator_LabelSelect_Batch_0_utzy75nd/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=

ar: creating VLabelSelect_Batch_0_LabelSelect_Batch_0__ALL.a


...

%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_0_rpi3x4bd/project_StreamingFCLayer_Batch_0/sol1/impl/verilog//StreamingFCLayer_Batch_0_StreamingFCLayer_Batch_0.v:78: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: Use "/* verilator lint_off STMTDLY */" and lint_on around source to disable this message.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_0_rpi3x4bd/project_StreamingFCLayer_Batch_0/sol1/impl/verilog//StreamingFCLayer_Batch_0_StreamingFCLayer_Batch_0.v:79: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_0_rpi3x4bd/project_StreamingFCLayer_Batch_0/sol1/impl/verilog//StreamingFCLayer_Batch_0_Matrix_Vector_Activa.v:4986: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFCLayer_Batch_0_rpi3x4bd/project_StreamingFCLayer_Batch_0/sol1/impl/verilog//StreamingFCLa

make: Entering directory '/tmp/finn_dev_zqc/pyverilator_Thresholding_Batch_0_0hwkiyvg'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_zqc/pyverilator_Thresholding_Batch_0_0hwkiyvg/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRAC

ar: creating VThresholding_Batch_0_Thresholding_Batch_0__ALL.a


make: Entering directory '/tmp/finn_dev_zqc/pyverilator_StreamingFCLayer_Batch_0_w64b9w28'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_zqc/pyverilator_StreamingFCLayer_Batch_0_w64b9w28/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -

ar: creating VStreamingFCLayer_Batch_0_StreamingFCLayer_Batch_0__ALL.a


The next step is to load the parent model and set the node attribute `model` in the StreamingDataflowPartition node (`sdp_node`). Afterwards the `exec_mode` is set in the parent model in each node.

In [20]:
# parent model
model_for_rtlsim = ModelWrapper(build_dir + "/tfc_w1_a1_dataflow_parent.onnx")
# reference child model
sdp_node = getCustomOp(model_for_rtlsim.graph.node[1])
sdp_node.set_nodeattr("model", build_dir + "/tfc_w1_a1_dataflow_child.onnx")

model_for_rtlsim = model_for_rtlsim.transform(SetExecMode("rtlsim"))

Because the necessary files for the emulation are already generated in Jupyter notebook [tfc_end2end_example](tfc_end2end_example.ipynb), in the next step the execution of the model can be done directly.

In [21]:
output_dict = oxe.execute_onnx(model_for_rtlsim, input_dict)
output_rtlsim = output_dict[list(output_dict.keys())[0]]

if np.isclose(output_rtlsim, output_golden, atol=1e-3).all():
    print("Results are the same!")
else:
    print("The results are not the same!")

The results are not the same!


### Emulation of stitched IP

Here we use the same procedure. First the child model is loaded, but in contrast to the layer-by-layer emulation, the metadata property `exec_mode` is set to "rtlsim" for the whole child model. When the model is integrated and executed in the last step, the verilog files of the stitched IP of the child model are used.

In [22]:
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.fpgadataflow.insert_fifo import InsertFIFO
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP

child_model = ModelWrapper(build_dir + "/tfc_w1_a1_dataflow_child.onnx")
child_model = child_model.transform(InsertDWC())
child_model = child_model.transform(InsertFIFO())
child_model = child_model.transform(GiveUniqueNodeNames())
child_model = child_model.transform(PrepareIP(test_fpga_part, target_clk_ns))
child_model = child_model.transform(HLSSynthIP())
child_model = child_model.transform(CreateStitchedIP(test_fpga_part, target_clk_ns))
child_model = child_model.transform(PrepareRTLSim())
child_model.set_metadata_prop("exec_mode","rtlsim")
child_model.save(build_dir + "/tfc_w1_a1_dataflow_child.onnx")

/workspace/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:154: UserWarning: Overriding input FIFO depth to 32
  warnings.warn("Overriding input FIFO depth to 32")
/workspace/finn/src/finn/transformation/fpgadataflow/insert_fifo.py:201: UserWarning: Overriding output FIFO depth to 32
  warnings.warn("Overriding output FIFO depth to 32")
/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:57: UserWarning: Using pre-existing code for Thresholding_Batch_0
  warnings.warn("Using pre-existing code for %s" % node.name)
/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:57: UserWarning: Using pre-existing code for StreamingFCLayer_Batch_0
  warnings.warn("Using pre-existing code for %s" % node.name)
/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:57: UserWarning: Using pre-existing code for StreamingFCLayer_Batch_1
  warnings.warn("Using pre-existing code for %s" % node.name)
/workspace/finn/src/finn/transformation/fpgadataflow/prepar

%Warning-WIDTH: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFIFO_6_6krxml_9/project_StreamingFIFO_6/sol1/impl/verilog//Q_srl.v:179: Operator ASSIGNDLY expects 80 bits on the Assign RHS, but Assign RHS's CONST '?32?bxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' generates 32 bits.
%Warning-WIDTH: Use "/* verilator lint_off WIDTH */" and lint_on around source to disable this message.
%Warning-WIDTH: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFIFO_6_6krxml_9/project_StreamingFIFO_6/sol1/impl/verilog//Q_srl.v:182: Operator ASSIGNDLY expects 4 bits on the Assign RHS, but Assign RHS's CONST '?32?bxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' generates 32 bits.
%Warning-WIDTH: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFIFO_6_6krxml_9/project_StreamingFIFO_6/sol1/impl/verilog//Q_srl.v:195: Operator ASSIGNDLY expects 80 bits on the Assign RHS, but Assign RHS's CONST '?32?bxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' generates 32 bits.
%Warning-WIDTH: /tmp/finn_dev_zqc/code_gen_ipgen_StreamingFIFO_6_6krxml_9/project_StreamingFIFO_6/s

%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch_0.v:65: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: Use "/* verilator lint_off STMTDLY */" and lint_on around source to disable this message.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch_0.v:66: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch.v:68: Unsupported: Ignoring delay on this delayed statement.
%Warning-WIDTH: /tmp/finn_dev_zqc/code_gen_ipgen_LabelSelect_Batch_0_gbpyxf2d/project_LabelSelect_Batch_0/sol1/impl/verilog//LabelSelect_Batch_0_LabelSelect_Batch.v:183: Operator ASSIGN expects 3 bits on the As

make: Entering directory '/tmp/finn_dev_zqc/pyverilator_StreamingFIFO_6_mqa69h53'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_zqc/pyverilator_StreamingFIFO_6_mqa69h53/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -falig

ar: creating VStreamingFIFO_6__ALL.a


...

ar: creating VStreamingFIFO_0__ALL.a


make: Entering directory '/tmp/finn_dev_zqc/pyverilator_Thresholding_Batch_0_4iszmcrj'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_zqc/pyverilator_Thresholding_Batch_0_4iszmcrj/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRAC

ar: creating VThresholding_Batch_0_Thresholding_Batch_0__ALL.a


make: Entering directory '/tmp/finn_dev_zqc/pyverilator_StreamingFCLayer_Batch_0_3tl81qcu'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_zqc/pyverilator_StreamingFCLayer_Batch_0_3tl81qcu/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -

ar: creating VStreamingFCLayer_Batch_0_StreamingFCLayer_Batch_0__ALL.a


In [24]:
# parent model
model_for_rtlsim = ModelWrapper(build_dir + "/tfc_w1_a1_dataflow_parent.onnx")
# reference child model
sdp_node = getCustomOp(model_for_rtlsim.graph.node[1])
sdp_node.set_nodeattr("model", build_dir + "/tfc_w1_a1_dataflow_child.onnx")

In [25]:
output_dict = oxe.execute_onnx(model_for_rtlsim, input_dict)
output_rtlsim = output_dict[list(output_dict.keys())[0]]

if np.isclose(output_rtlsim, output_golden, atol=1e-3).all():
    print("Results are the same!")
else:
    print("The results are not the same!")

make: Entering directory '/tmp/finn_dev_zqc/pyverilator_ipstitched_tpznhv6q'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /tmp/finn_dev_zqc/pyverilator_ipstitched_tpznhv6q/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -W

ar: creating Vfinn_design_wrapper__ALL.a
